# Ensemble of ensembles - model stacking

* **Ensemble with different types of classifiers**:
  * Different types of classifiers (E.g., logistic regression, decision trees, random forest, etc.) are fitted on the same training data
  * Results are combined based on either
    * majority voting (classification) or
    * average (regression)
  

* **Ensemble with a single type of classifier**:
  * Bootstrap samples are drawn from training data
  * With each bootstrap sample, model (E.g., Individual model may be decision trees, random forest, etc.) will be fitted
  * All the results are combined to create an ensemble.
  * Suitabe for highly flexible models that is prone to overfitting / high variance.

***

## Combining Method

* **Majority voting or average**:
  * Classification: Largest number of votes (mode)
  * Regression problems: Average (mean).
  
  
* **Method of application of meta-classifiers on outcomes**:
  * Binary outcomes: 0 / 1 from individual classifiers
  * Meta-classifier is applied on top of the individual classifiers.
  
  
* **Method of application of meta-classifiers on probabilities**:
  * Probabilities are obtained from individual classifiers.
  * Applying meta-classifier
  

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
df = pd.read_csv("../data/WA_Fn-UseC_-HR-Employee-Attrition.csv")
num_col = list(df.describe().columns)
col_categorical = list(set(df.columns).difference(num_col))
remove_list = ['EmployeeCount', 'EmployeeNumber', 'StandardHours']
col_numerical = [e for e in num_col if e not in remove_list]
attrition_to_num = {'Yes': 0,
                    'No': 1}
df['Attrition_num'] = df['Attrition'].map(attrition_to_num)
col_categorical.remove('Attrition')
df_cat = pd.get_dummies(df[col_categorical])
X = pd.concat([df[col_numerical], df_cat], axis=1)
y = df['Attrition_num']

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [5]:
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, roc_auc_score
def print_score(clf, X_train, X_test, y_train, y_test, train=True):
    '''
    v0.1 Follow the scikit learn library format in terms of input
    print the accuracy score, classification report and confusion matrix of classifier
    '''
    lb = preprocessing.LabelBinarizer()
    lb.fit(y_train)
    if train:
        '''
        training performance
        '''
        print("Train Result:\n")
        print("accuracy score: {0:.4f}\n".format(accuracy_score(y_train, clf.predict(X_train))))
        print("Classification Report: \n {}\n".format(classification_report(y_train, clf.predict(X_train))))
        print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_train, clf.predict(X_train))))
        print("ROC AUC: {0:.4f}\n".format(roc_auc_score(lb.transform(y_train),
                                                        lb.transform(clf.predict(X_train)))))

        #cv_res = cross_val_score(clf, X_train, y_train, cv=10, scoring='accuracy')
        #print("Average Accuracy: \t {0:.4f}".format(np.mean(cv_res)))
        #print("Accuracy SD: \t\t {0:.4f}".format(np.std(cv_res)))

    elif train==False:
        '''
        test performance
        '''
        res_test = clf.predict(X_test)
        print("Test Result:\n")
        print("accuracy score: {0:.4f}\n".format(accuracy_score(y_test, clf.predict(X_test))))
        print("Classification Report: \n {}\n".format(classification_report(y_test, clf.predict(X_test))))
        print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_test, clf.predict(X_test))))
        print("ROC AUC: {0:.4f}\n".format(roc_auc_score(lb.transform(y_test), lb.transform(res_test))))


## Model 1: Decision Tree

In [6]:
from sklearn.tree import DecisionTreeClassifier

In [7]:
tree_clf = DecisionTreeClassifier()
tree_clf.fit(X_train, y_train)

DecisionTreeClassifier()

In [8]:
print_score(tree_clf, X_train, X_test, y_train, y_test, train=True)
print_score(tree_clf, X_train, X_test, y_train, y_test, train=False)

Train Result:

accuracy score: 1.0000

Classification Report: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       167
           1       1.00      1.00      1.00       935

    accuracy                           1.00      1102
   macro avg       1.00      1.00      1.00      1102
weighted avg       1.00      1.00      1.00      1102


Confusion Matrix: 
 [[167   0]
 [  0 935]]

ROC AUC: 1.0000

Test Result:

accuracy score: 0.7935

Classification Report: 
               precision    recall  f1-score   support

           0       0.45      0.37      0.41        70
           1       0.86      0.89      0.88       298

    accuracy                           0.79       368
   macro avg       0.65      0.63      0.64       368
weighted avg       0.78      0.79      0.79       368


Confusion Matrix: 
 [[ 26  44]
 [ 32 266]]

ROC AUC: 0.6320



## Model 2: Random Forest

In [9]:
from sklearn.ensemble import RandomForestClassifier

In [10]:
rf_clf = RandomForestClassifier(n_estimators=100)
rf_clf.fit(X_train, y_train)

RandomForestClassifier()

In [11]:
print_score(rf_clf, X_train, X_test, y_train, y_test, train=True)

Train Result:

accuracy score: 1.0000

Classification Report: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       167
           1       1.00      1.00      1.00       935

    accuracy                           1.00      1102
   macro avg       1.00      1.00      1.00      1102
weighted avg       1.00      1.00      1.00      1102


Confusion Matrix: 
 [[167   0]
 [  0 935]]

ROC AUC: 1.0000



In [12]:
print_score(rf_clf, X_train, X_test, y_train, y_test, train=False)

Test Result:

accuracy score: 0.8451

Classification Report: 
               precision    recall  f1-score   support

           0       0.78      0.26      0.39        70
           1       0.85      0.98      0.91       298

    accuracy                           0.85       368
   macro avg       0.82      0.62      0.65       368
weighted avg       0.84      0.85      0.81       368


Confusion Matrix: 
 [[ 18  52]
 [  5 293]]

ROC AUC: 0.6202



In [13]:
en_en = pd.DataFrame()

In [14]:
tree_clf.predict_proba(X_train)

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [1., 0.]])

In [15]:
en_en['tree_clf'] = pd.DataFrame(tree_clf.predict_proba(X_train))[1]
en_en['rf_clf'] =  pd.DataFrame(rf_clf.predict_proba(X_train))[1]
col_name = en_en.columns
en_en = pd.concat([en_en, pd.DataFrame(y_train).reset_index(drop=True)], axis=1)

In [16]:
en_en.head()

,tree_clf,rf_clf,Attrition_num
0,1.0,0.87,1
1,0.0,0.20,0
2,1.0,0.99,1
3,1.0,0.92,1
4,1.0,0.84,1


In [17]:
tmp = list(col_name)
tmp.append('ind')
en_en.columns = tmp

In [18]:
en_en.head()

,tree_clf,rf_clf,ind
0,1.0,0.87,1
1,0.0,0.20,0
2,1.0,0.99,1
3,1.0,0.92,1
4,1.0,0.84,1


# Meta Classifier

In [19]:
from sklearn.linear_model import LogisticRegression

In [20]:
m_clf = LogisticRegression(fit_intercept=False, solver='lbfgs')

In [21]:
m_clf.fit(en_en[['tree_clf', 'rf_clf']], en_en['ind'])

LogisticRegression(fit_intercept=False)

In [22]:
en_test = pd.DataFrame()

In [23]:
en_test['tree_clf'] = pd.DataFrame(tree_clf.predict_proba(X_test))[1]
en_test['rf_clf'] =  pd.DataFrame(rf_clf.predict_proba(X_test))[1]
col_name = en_en.columns
en_test['combined'] = m_clf.predict(en_test[['tree_clf', 'rf_clf']])

In [24]:
col_name = en_test.columns
tmp = list(col_name)
tmp.append('ind')

In [25]:
tmp

['tree_clf', 'rf_clf', 'combined', 'ind']

In [26]:
en_test = pd.concat([en_test, pd.DataFrame(y_test).reset_index(drop=True)], axis=1)

In [27]:
en_test.columns = tmp

In [28]:
print(pd.crosstab(en_test['ind'], en_test['combined']))

combined   0    1
ind              
0         26   44
1         32  266


In [29]:
print(round(accuracy_score(en_test['ind'], en_test['combined']), 4))

0.7935


In [30]:
print(classification_report(en_test['ind'], en_test['combined']))

              precision    recall  f1-score   support

           0       0.45      0.37      0.41        70
           1       0.86      0.89      0.88       298

    accuracy                           0.79       368
   macro avg       0.65      0.63      0.64       368
weighted avg       0.78      0.79      0.79       368



***